## Общая схема квазиньютоновских методов

```python
def QuasiNewtonMethod(f, x0, epsilon, **kwargs):
    
    x = x0
    
    H = I
    
    while True:
        
        h = -H.dot(grad_f(x))
        
        if StopCriterion(x, f, h, **kwargs) < epsilon:
            
            break
            
        alpha = SelectStepSize(x, h, f, **kwargs)
            
        x = x + alpha * h
            
        H = UpdateH(H, f(x), grad_f(x))
            
    return x
```

## DFP (Davidon-Fletcher-Powell)

$$
B_{k+1} = (I - \rho_k y_k s^{\top}_k)B_k(I - \rho_k s_ky^{\top}_k) + \rho_k y_k y^{\top}_k,
$$

где $\rho_k = \dfrac{1}{y^{\top}_k s_k}$,

или с помощью формулы Шермана-Морисона-Вудбери

$$
B^{-1}_{k+1} = H_{k+1} = H_k - \dfrac{H_ky_k y_k^{\top}H_k}{y^{\top}_kH_ky_k} + \dfrac{s_ks^{\top}_k}{y^{\top}_ks_k}
$$

**Вопрос:** какой ранг у разности матриц $B_{k+1} (H_{k+1})$ и $B_{k} (H_{k})$?

### Вывод

Общая идея квазиньютоновских методов: 

вместо полного пересчёта гессиана на каждой итерации обновлять 

текущую его аппроксимацию с помощью легко вычислимого 

преобразования

In [2]:
import numpy as np
import numpy.linalg as ln
import scipy as sp
import scipy.optimize


# Функция
def f(x):
    return x[0]**2 + x[1]**2- 3*x[1]


# Производная
def f1(x):
    return np.array([2 * x[0],2*x[1] - 3])


def dfp_method(f, fprime, x0, maxiter=None, epsi=10e-3):
    
    if maxiter is None:
        maxiter = len(x0) * 200

    k = 0
    gfk = fprime(x0)
    N = len(x0)
    I = np.eye(N, dtype=int)
    Hk = I
    xk = x0
   
    while ln.norm(gfk) > epsi and k < maxiter:
        
        pk = -np.dot(Hk, gfk)
        

        line_search = sp.optimize.line_search(f, f1, xk, pk)
        alpha_k = line_search[0]
        
        xkp1 = xk + alpha_k * pk
        sk = xkp1 - xk
        xk = xkp1
        
        gfkp1 = fprime(xkp1)
        yk = gfkp1 - gfk
        gfk = gfkp1
        
        k += 1
        
        ro = 1.0 / (np.dot(yk, sk))
        ro1 = 1.0 / (np.dot(np.dot(Hk, yk), yk))
        ro2 =  yk[:, np.newaxis] * yk[np.newaxis, :]
        A = np.dot(Hk, np.dot(ro2, Hk))
        Hk= Hk-np.dot(ro1,A)+ (ro * sk[:, np.newaxis] * sk[np.newaxis, :])
        
    return (xk, k)


result, k = dfp_method(f, f1, np.array([1, 1]))# начальная точка 1 1 

print('Наилучшая точка минимума: %s' % (result))
print('Количество итераций: %s' % (k))

Наилучшая точка минимума: [0.  1.5]
Количество итераций: 1
